In [ ]:
import os
from typing import TypedDict, Literal

from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.chains import LLMMathChain
from langgraph.graph import StateGraph, END

In [ ]:
# --- Environment and LLM Setup ---
os.environ["SERPAPI_API_KEY"] = "42a63592c032fc7a97f972c068c18c2055f7409b1886d53270e4e77d089fd71f"

llm = ChatOpenAI(
    api_key="not-needed",
    base_url="http://localhost:1234/v1",
    model="local-model",
    temperature=0,
    streaming=True
)

# --- Tool Definitions ---
search_tool = SerpAPIWrapper()
math_tool = LLMMathChain.from_llm(llm=llm, verbose=False)

In [ ]:
# --- State Definition ---
# Updated state to include a route for travel and a result.
class AgentState(TypedDict):
    topic: str
    explanation: str
    summary: str
    calculator_result: str
    travel_result: str
    route: Literal["research", "calculate", "travel"] # The planner's decision

In [ ]:
def planner_agent(state: AgentState) -> dict:
    """
    This agent decides whether the user's topic is a math, research, or travel question.
    """
    print("---PLANNER---")
    topic = state["topic"]
    
    prompt = ChatPromptTemplate.from_template(
        """You are a planner agent. Your job is to determine the best path to take based on the user's query.
        
        Query: {topic}
        
        Based on the query, is this a math problem, a travel-related query (like finding hotels or flights), or a general research question?
        Respond with 'calculate', 'travel', or 'research'.
        """
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": topic})
    decision = result.content.strip().lower()
    
    print(f"Planner's Decision: {decision}")
    if "calculate" in decision:
        return {"route": "calculate"}
    elif "travel" in decision:
        return {"route": "travel"}
    else:
        return {"route": "research"}

In [ ]:
def researcher_agent(state: AgentState) -> dict:
    """
    This agent uses a web search tool to find information on a topic and then explains it.
    """
    print("---RESEARCHER (with SerpApi Web Search)---")
    topic = state["topic"]
    
    search_results = search_tool.run(topic)
    print(f"Search Results:\n{search_results}")
    
    prompt = ChatPromptTemplate.from_template(
        """You are a helpful research assistant. Based on the following search results,
        provide a brief, easy-to-understand explanation of the topic: {topic}.

        Search Results:
        {search_results}
        """
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})
    
    print(f"Researcher's Explanation:\n{result.content}")
    return {"explanation": result.content}

In [ ]:
def calculator_agent(state: AgentState) -> dict:
    """
    This agent uses an LLM Math tool to solve a math problem.
    """
    print("---CALCULATOR---")
    topic = state["topic"]
    
    result = math_tool.invoke({"question": topic})
    answer = result.get('answer', 'Could not calculate answer.')
    
    print(f"Calculator's Result: {answer}")
    return {"calculator_result": answer}

In [ ]:
def travel_planner_agent(state: AgentState) -> dict:
    """
    This agent searches for travel-related information like hotels or flights.
    """
    print("---TRAVEL PLANNER---")
    topic = state["topic"]
    
    # For this example, we'll just use the general search tool.
    # A real-world application would use a dedicated travel API.
    search_results = search_tool.run(f"Find information about: {topic}")
    
    print(f"Travel Search Results:\n{search_results}")
    
    prompt = ChatPromptTemplate.from_template(
        """You are a helpful travel assistant. Summarize the following information for the user's travel query about '{topic}'.

        Search Results:
        {search_results}
        """
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": topic, "search_results": search_results})
    
    print(f"Travel Planner's Response:\n{result.content}")
    return {"travel_result": result.content}

In [ ]:
def summarizer_agent(state: AgentState) -> dict:
    """
    This agent takes an explanation and summarizes it in one sentence.
    """
    print("---SUMMARIZER---")
    explanation = state["explanation"]
    
    prompt = ChatPromptTemplate.from_template(
        "You are a summarization expert. Condense the following text into a single, concise sentence:\n\n{explanation}"
    )
    
    chain = prompt | llm
    result = chain.invoke({"explanation": explanation})
    
    print(f"Summarizer's Output:\n{result.content}")
    return {"summary": result.content}

In [ ]:
# --- Conditional Routing Function ---
def should_route(state: AgentState) -> Literal["research", "calculate", "travel"]:
    """This function is the decision point for our conditional edge."""
    return state["route"]

In [ ]:
# --- Graph Definition ---

workflow = StateGraph(AgentState)

# Add all the agent functions as nodes
workflow.add_node("planner", planner_agent)
workflow.add_node("researcher", researcher_agent)
workflow.add_node("calculator", calculator_agent)
workflow.add_node("travel_planner", travel_planner_agent)
workflow.add_node("summarizer", summarizer_agent)

# Set the entry point to the planner
workflow.set_entry_point("planner")

# Add the conditional edge for the planner
workflow.add_conditional_edges(
    "planner",
    should_route,
    {
        "research": "researcher",
        "calculate": "calculator",
        "travel": "travel_planner",
    },
)

# Define the standard edges
workflow.add_edge("researcher", "summarizer")
workflow.add_edge("summarizer", END)
workflow.add_edge("calculator", END)
workflow.add_edge("travel_planner", END) # Travel result is final

# Compile the graph
app = workflow.compile()

In [ ]:
# --- Main Execution Block ---
if __name__ == "__main__":
    topic = input("Please enter a topic, math problem, or travel query: ")

    inputs = {"topic": topic}
    final_state = app.invoke(inputs)
    
    print("\n--- FINAL RESULT ---")
    # The final result depends on which path was taken
    if final_state.get('summary'):
        print(final_state['summary'])
    elif final_state.get('calculator_result'):
        print(final_state['calculator_result'])
    elif final_state.get('travel_result'):
        print(final_state['travel_result'])
    else:
        print("An unexpected error occurred.")